# Used cars market facts

As an aspiring Data Analyst, (or scientist ? or machine learning engineer ? work in progress ...) I am fascinated by the data. When I had to make a project, as a part of my training with Jovian [[Data Analysis with Python: Zero to Pandas]](zerotopandas.com), I tried to combine my passion for data and my all-time favorite topic : Cars !<br>
As a car-enthousiast, I like working on cars. I'm often in the market for used cars and parts. The buying process involves lots of research and comparison. The ultimate goal is to get the best deal, using all the available tools.<br>
___But what is a good deal ?___<br>
While many parameters influence a car buyer, the decision always involves _emotion_. The first thing to consider before taking any __decision__ is to have as much __good quality data__ as possible. We have then to perform analysis and extract informations.<br>
In this notebook, I study a public dataset available on [KAGGLE](kaggle.com). The dataset is scraped from __Craiglist__, a very popular classified ads in the US, and north America. It contains informations from around 450k car classified ads. It is updated every 18 months and contains a fair amount of properties for each transaction (26 columns). More details can be found on [this link](https://www.kaggle.com/austinreese/craigslist-carstrucks-data).


In [ ]:
!pip install opendatasets pandas --quiet


Let's begin by downloading the data, and listing the files within the dataset.

In [ ]:
dataset_url = 'https://www.kaggle.com/austinreese/craigslist-carstrucks-data'
import opendatasets as od
od.download(dataset_url)

The dataset has been downloaded and extracted.

In [ ]:
data_dir = './craigslist-carstrucks-data'
import os
os.listdir(data_dir)

## Data Preparation and Cleaning

First step is to explore the file we've just downloaded. Let's use pandas to open this csv.

In [ ]:
import pandas as pd
import numpy as np
cars_df = pd.read_csv(data_dir+'/vehicles.csv')

Let us have a first look at this dataset :

In [ ]:
cars_df.info()

This Dataframe has 25 columns of various type. It contains 441396 rows. Each row represents a car ad with all the available properties. Some columns are full (like price, region, etc) while some other columns are incomplete (exple : VIN, size etc). Let's explore the columns:

In [ ]:
cars_df.columns

The next step is to describe each column, and handle missing or wrong data. First we can remove all the unnecessary columns (for our project):   **'url', 'region_url', 'VIN', 'image_url', 'description', 'lat', 'long', 'size'**

In [ ]:
remove_columns = ['url', 'region_url', 'VIN', 'image_url', 'description', 'lat', 'long', 'size']
for column in remove_columns:
    cars_df.drop(column, axis = 1, inplace= True)

The next cell, shows the number of missing elements for each column:

In [ ]:
for column in cars_df.columns:
    print('Column :', column)
    print('Number of Nan', cars_df[column].isna().sum())
    print(cars_df[column].describe())
    print('****************************')

The number of empty cells varies among columns. For this project, the main columns are **price** and **year**. According to the previous cell, no price row is empty ! (we still need to look for outliers). The **year** column contains some empty cells, that we choose to simply drop.

In [ ]:
cars_df.dropna(subset=['year'], axis = 0, inplace=True)

Categorical data could be useful in some analysis. Here, two columns are created in order to classify the price, and the odometer reading: 

In [ ]:
bins=[0,10000,20000,30000,40000]
cars_df['price_cat'] = pd.cut(cars_df['price'] , bins=bins , labels=[1,2,3,4])

In [ ]:
bins = [0,25000,50000,100000,np.inf]    
cars_df['odo_cat'] = pd.cut(cars_df['odometer'], bins = bins, labels = [1,2,3,4])

## Exploratory Analysis and Visualization

In this section, we will consider some columns and study their statistical properties.  
Let's begin by importing`matplotlib.pyplot` and `seaborn`.

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (9, 5)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In this graph, we will determine the number of cars from each manufacturer. The plot will show the top 15 : 

In [ ]:
cars_manufacturers = cars_df.groupby('manufacturer').id.count().sort_values(ascending=False).head(15)
plt.figure(figsize=(30, 6))
plt.bar(cars_manufacturers.index, cars_manufacturers.values);
plt.ylabel('Number of cars');
plt.title('Number of cars by manufacturer');

This dataset is scraped from a US website. The top selling brands are domestic (Ford, GM) and japanese (Toyota, Honda).<br>
In the next plot, we will consider the geographical distribution of the data, based on the **state** column.

In [ ]:
cars_state = cars_df.groupby('state').id.count().sort_values(ascending=False).head(15)
plt.figure(figsize=(30, 6))
plt.bar(cars_state.index, cars_state.values);
plt.ylabel('Number of cars');
plt.title('Number of cars by state');

It appears that California has the most data, which is expected due to its large population. That being said, the dataset is balanced when considering the other states.<br>
The number of used cars for sale is related to the age of the vehicule. We can see this relationship through the column **year**. We will not consider cars made prior to 1980 in order to get a better graph.

In [ ]:
cars_year = cars_df[cars_df['year']>1980].groupby('year').id.count()
plt.figure(figsize=(30, 6))
plt.bar(cars_year.index, cars_year.values);
plt.xlabel('Year')
plt.ylabel('Number of cars');
plt.title('Number of cars by year');

We can see a drop in the number of available cars in 2009. Could it be related to the drop in new cars sales during the previous year (subprime crisis) and the bankruptcy of car manufacturers ? The same reasoning could be applied to 2020 with the Covid-19 pandemic. More reseach is needed to confirm this hypothesis.<br>
Let establish the relationship between the average price of cars and some other factors including : Year, Manufacturer and State. We will consider cars made after 1980 to ease comparison. The outliers in term of car prices will be removed as well. 

In [ ]:
cars_df2 =  cars_df[cars_df['year']>1980] # cars newer than 1980
cars_df2 = cars_df2[cars_df2.price <100000] # cars cost less than 100k
cars_df2 = cars_df2[cars_df2.odometer <1000000] # odometer less than 1000000Miles
price_vs_year = cars_df2.groupby('year').price.mean()
price_vs_year 
plt.figure(figsize=(30, 6))
plt.plot(price_vs_year.index, price_vs_year .values);
plt.xlabel('Year')
plt.ylabel('Average price');
plt.title('Average price of cars by year');

In [ ]:
price_vs_manufacturer = cars_df2.groupby('manufacturer').price.mean().sort_values(ascending=False).head(20)
plt.figure(figsize=(36, 6))
plt.bar(price_vs_manufacturer.index, price_vs_manufacturer.values);
plt.xlabel('Year')
plt.ylabel('Average price');
plt.title('Average price by manufacturer [top 20]');

Obviously newer cars are more expensive than older cars, unless we consider very old cars which is collectors territory. The price by manufacturer graph shows some interesting facts. Tesla average price is higher than some luxury cars. This is probably due to the fact that Tesla is a new car manufacturer. The inventory of electric cars for sale is still recent and prices are higher.  

In [ ]:
price_vs_state = cars_df2.groupby('state').price.mean().sort_values(ascending=False)
plt.figure(figsize=(36, 6))
plt.bar(price_vs_state.index, price_vs_state.values);
plt.xlabel('Year')
plt.ylabel('Mean price');
plt.title('Average car price by State');

We can observe that the prices can vary significantly between states. The local taxes, emission standards, and type of vehicules can affect greatly the mean price. In the top 10 for instance, we could find many rural states, where the most common vehicules are full-size trucks.

A scatter plot will show the relationship between mileage, age and sell price of used cars.


In [ ]:
plt.figure(figsize=(24, 10));
sns.scatterplot(x='year', y='odometer', hue='price', data=cars_df2, s=1000)
plt.xlabel('Year')
plt.ylabel('Mileage')

The darker colors refer to higher prices. We can observe that in general, the price is higher for recent cars, with low mileage. But by looking at the graph, we can spot multiple exceptions. This could be related to outliers, or to specific car models / makes. As we said earlier, the car purshase process is very affected by emotion. Mileage and year of manufacture are not the only factors to consider.

Let's plot the different fuel types available in the US market, based on data from the last 40 years.

In [ ]:
fuel_types = cars_df2.groupby('fuel').id.count()
plt.figure(figsize=(36, 6))
plt.bar(fuel_types.index, fuel_types.values);
plt.ylabel('Number of cars');
plt.title('Type of fuel');

No surprise ! the gasoline-powererd cars are the most popular cars in the US, including trucks. This is very different then other parts of the world, where diesel is more frequent. 

## Asking and Answering Questions

For the next section, we will use the cars_df2 dataset, wich contains data from 1980 to today.  
Q1 : what is the most popular car model in the USA ? are there any differences between states ?

In [ ]:
us_top_cars = cars_df2.groupby('model').id.count().sort_values(ascending=False).head(15)
us_top_cars

The Ford F-150 is the most sold car in the US in general, the second being another truck (Silverado 1500). Let's see if this varies between states:

In [ ]:
states = cars_df2.state.unique()
# print the top 5 selling vehicules per state
for state in states :
    h = cars_df2[cars_df2['state']==state].groupby('model').id.count().sort_values(ascending=False).head(5)
    print('Top selling cars in ({}) are {}, with {} sales.'.format(state, list(h.index), list(h.values)))

It seems like F-150 is the most sold car (truck) in most of the states.<br> Some exceptions exist still like in **Nevada (NV), Delaware (DE), Connecticut (CT)**. According to the dataset, one of the most sold vehicule in **Rhodes Island (RI)** is Freightliner M2 (a large size truck). This result requires more investigation to confirm. 

Q2 : What makes/models are used more intensively ?<br>
In order to measure this quality, we could find the average mileage per year for each make/model.

In [ ]:
cars_df2['year_mileage']=cars_df2.odometer/(2020-cars_df2['year'][cars_df2.year !=2020])
heavy_duty_models = cars_df2.groupby('model').year_mileage.mean().sort_values(ascending=False).head(15)
heavy_duty_manufacturer = cars_df2.groupby('manufacturer').year_mileage.mean().sort_values(ascending=False).head(15)
print(heavy_duty_models)
print('\n')
print(heavy_duty_manufacturer)

The dataset column 'model' requires more improvment/processing. This will allow more precision and improve the precision of visualisations/analysis. The most used vehicules are commercial trucks. When it comes to manufacturer, the results still consists in trucks mainly.

Q3 : which drivetrains are more frequent per state ?

In [ ]:
states = cars_df2.state.unique()
for state in states :
    h = cars_df2[cars_df2['state']==state].groupby('drive').id.count().sort_values(ascending=False).head(1)
    print('The most drivetrain in ({}) is {}.'.format(state, list(h.index)))

4wd (or 4x4 or awd) is very popular in the US. Some states have more emission regulations and fuel-economy taxes. In **CA** for instance, FWD (front wheel drive) are more comon due to higher fuel-efficiency.

Q4 : Which car brand depreciates the most ?<br>
For each manufacturer, we have to measure the drop in the average price, per year. We will consider the depreciation during the first 5 years.

In [ ]:
deprec_df=cars_df[['manufacturer', 'year', 'price']].dropna(subset=['manufacturer'], axis = 0)
years = [2019,2014]
mean_price=[]
for year in years:
    mean_price.append([year,deprec_df[deprec_df['year']==year].groupby('manufacturer').price.mean()])


In [ ]:
manufacturers = [i for i in mean_price[0][1].index if i in mean_price[1][1].index]
depreciation =[]
for manufacturer in manufacturers:
    delta = (mean_price[0][1][manufacturer]-mean_price[1][1][manufacturer])/mean_price[0][1][manufacturer]
    depreciation.append(delta)
    print('manufacturer : {} depreciation in 5 years : {:.2f}%'.format(manufacturer,delta*100))

print('The most depreciating brand is {}'.format(manufacturers[np.argmax(depreciation)]))

Luxury and european cars, such as Mercedes-Benz and BMW suffer from higher depreciation than japanese cars. Tesla has a low depreciation rate, like most electric cars. Surprisingly, Harley-Davidson has a negative score. This is due mainly to the low number of Harley-Davidson bikes in this dataset. In this case, data is not representative.

In [ ]:
cars_df[cars_df['manufacturer']=='harley-davidson'].price.count()

Only 110 bikes in 440k cars ad.

## Inferences and Conclusion

We have seen in this analysis that the price of used cars is affected by multiple factors. These factors include : age, mileage, location, brand, etc.<br>
The used dataset in this project is of a good quality in general, but some columns need more _cleaning_ in order to get better precision.<br>
Many intuitions could be made, based on the data we used here. We confirmed also some hypothesis, in particular about depreciation, and the car market in general, in the USA.

## References and Future Work

This analysis was a course project with [Jovian](https://jovian.ai/learn/data-analysis-with-python-zero-to-pandas).  

The dataset [offered by AustinReese] was downloaded from [Kaggle](https://www.kaggle.com/austinreese/craigslist-carstrucks-data). It is generated and updated every 18 months, via [this application](https://github.com/AustinReese/UsedVehicleSearch).<br>
Many potential applications could be imagined, based on the results in this analysis. A prediction model of car prices based on the ad data could be made using machine learning or deep networks. A more in depth knowledge is required in order to select the best features to use in order to get the best prediction model. <br>
We could also use this data to study the market evolution, especially the new car segment : **electric cars**. The impact of major economical events can be correlated to the market evolution as well. 